In [ ]:
import os
from typing import Dict, List

import pandas as pd

from graphml_class.clean import clean_graph
from graphml_class.palette import CATEGORICAL_PALETTE

Part 3 - Graph Analytics with Cypher and Neo4j
==============================================

<center><img src="images/Neo4j-logo_color.png" width="500px" /></center>

<br />

In Part 3 of our course, we will use Neo4j and Bloom to learn Graph Analytics.

The following resources will assist you in learning Cypher to query Neo4j and other graph databases via [openCypher](https://opencypher.org/) ([github](https://github.com/opencypher)):

* [Neo4j Cypher Manual - Overview](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_overview/)
* [Neo4j Cypher Manual - Cypher and Neo4j](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_neo4j/)
* [Neo4j 5 Cypher Cheat Sheet](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise/)
* [Neo4j Graph Academy Cypher Fundamentals - 1 Hour Video](https://graphacademy.neo4j.com/courses/cypher-fundamentals/)
* [Neo4j Cypher Manual PDF](https://neo4j.com/docs/pdf/neo4j-cypher-manual-5.pdf)
* [Neo4j Getting Started - Query a Neo4j database using Cypher](https://neo4j.com/docs/getting-started/cypher-intro/)
* [Bite-Sized Neo4j for Data Scientists](https://neo4j.com/video/bite-sized-neo4j-for-data-scientists/) [[github code](https://github.com/cj2001/bite_sized_data_science)]
* [Using Neo4j from Python](https://neo4j.com/developer/python/)

We are going to call Neo4j from Python in this notebook. For simplicity's sake in setting up the course's software, I build a docker image and we run Neo4j Community on Docker via our [docker-compose.yml](docker-compose.yml) file. Docker only supports Neo4j Community unless you get a license (checkout [Neo4j for Startups](https://neo4j.com/startups/)) If you install [Neo4j Desktop](https://neo4j.com/download/) - which comes with a [developers' license]() to [Neo4j Enterprise]() - you can use [Neo4j Bloom](https://neo4j.com/product/bloom/). It is snazzy! :) It can visualize schemas and query results.

# Read Learning Cypher

I flash-learned Cypher based on experience with other query languages in a few hours by reading [Learning Cypher](https://learning.oreilly.com/library/view/learning-cypher/9781783287758/) by [Onofrio Panzarino](https://www.linkedin.com/in/onofriopanzarino/). While the resources above are useful, I recommend you buy this book to continue learning to do graph analytics with Neo4j.

# Connecting to Neo4j from Python

We are going to be using two ways to query Cypher.

1) The [neo4j](https://pypi.org/project/neo4j/) PyPi module [[github](https://github.com/neo4j/neo4j-python-driver)], [[Neo4j docs](https://neo4j.com/docs/api/python-driver/current/)]
2) Neo4j's Graphistry Integration [[example notebook](https://github.com/graphistry/pygraphistry/blob/master/demos/demos_databases_apis/neo4j/official/graphistry_bolt_tutorial_public.ipynb)]

We're going to use one or the other as the instructor prefers, depending on how important it is to visualize our results in a table, chart or as a network visualizaton using [Graphistry](https://graphistry.com).

## Connecting Using `neo4j` PyPi Package

The documentation for this library can be out of date, so I'd recommend consulting the source code on Github at [neo4j/neo4j-python-driver](https://github.com/neo4j/neo4j-python-driver) directly, if you run into trouble. I could not import a `neo4j.GraphDatabase` object... kind of an oversight :) It happens, enterprise software is difficult.

This setup uses Docker, which publishes port 7687 to your local machine. Check out the `neo4j` service defined in [docker-compose.yml](docker-compose.yml) if you're curious.

In [ ]:
from neo4j import GraphDatabase, RoutingControl

# Docker sends port 7687 to localhost
URI = "neo4j://localhost:7687"
# We have no authentication setup... in practice you may ;)
# AUTH = ("neo4j", "neo4j")

driver = GraphDatabase.driver(URI)
driver

## A `cypher(query)` Utility

The `neo4j` API is a little funny, so we write our own utility method `cypher(query)` to run queries and display a `pd.DataFrame`.

In [ ]:
def cypher(query: str) -> pd.DataFrame:
    """Run a cypher query"""
    
    def tx_query(tx):
        result = tx.run(query)
        return result.data()

    with driver.session() as session:
        data = session.execute_read(tx_query)

    return pd.DataFrame(data)

### Hello World: What Node Labels? Edge Types?

There are... a number. We can use a driver session to run a query in a transaction and get results back that we can convert to a pandas `pd.DataFrame`. Cool!

In [ ]:
cypher("CALL db.labels()")

In [ ]:
cypher("CALL db.relationshipTypes()")

In [ ]:
pd.set_option('display.width', 1000)

cypher("MATCH (n) RETURN DISTINCT labels(n) as Labels, COUNT(*) as Count ORDER BY Count DESC")

## Using Graphistry via `Plotter.cypher()`

Graphistry has the ability to call Neo4j directlyl using its `graphistry.cypher()` method. First you need to initialize it using Graphistry bolt via `graphistry.register(bolt=NEO4J)`, which I'll show you below :)

In [ ]:
import graphistry

In [ ]:
# Environment variable setup
GRAPHISTRY_USERNAME = os.getenv("GRAPHISTRY_USERNAME")
GRAPHISTRY_PASSWORD = os.getenv("GRAPHISTRY_PASSWORD")

In [ ]:
graphistry.register(
    api=3,
    username=GRAPHISTRY_USERNAME,
    password=GRAPHISTRY_PASSWORD,
)

In [ ]:
# Configuration for Graphistry
GRAPHISTRY_PARAMS = {
    "play": 1000,
    "pointOpacity": 0.7,
    "edgeOpacity": 0.3,
    "edgeCurvature": 0.3,
    "showArrows": True,
    "gravity": 0.15,
    "showPointsOfInterestLabel": False,
    "labels": {
        "shortenLabels": False,
    },
    "scalingRatio": 0.5,
}
FAVICON_URL = "https://graphlet.ai/assets/icons/favicon.ico"
LOGO = {"url": "https://graphlet.ai/assets/Branding/Graphlet%20AI.svg", "dimensions": {"maxWidth": 100, "maxHeight": 100}}

In [ ]:
NEO4J = {
    'uri': "bolt://heracles:7687", 
}

graphistry.register(bolt=NEO4J)

In [ ]:
# Get the OFAC nodes and render them
g = graphistry.cypher(
""""""
)

In [ ]:
g2 = g.bind(
    point_title="caption",
    point_label="caption",
)

In [ ]:
g3 = (
    g2
    .addStyle(
        page={
            "title": "Some Open Data",
            "favicon": FAVICON_URL
        },
        logo=LOGO,
    )
    .settings(
        url_params=GRAPHISTRY_PARAMS,
        height=800,
    )
)
g3.plot()

## Describe Our Network

When you start working with a new graph database, you need to get oriented by describing the schema, then the network, then sampling some data and finally with some exploratory data analysis. From there you know enough to begin your own workflows.

### Describing our Property Graph Model

Let's start by describing our network. What kinds of nodes are there?

In [ ]:
query = """"""

def distinct_labels_query(tx):
    result = tx.run(query)
    return pd.DataFrame.from_dict(result.data(), orient="columns")

with driver.session() as session:
    df = session.execute_read(distinct_labels_query)

df.p.apply(pd.Series)

In [ ]:
df.p.apply(pd.Series)[0].apply(pd.Series)